# External Data 

This chapter described the various external data that is shipped with CASA and periodically updated.



***





### CASA Data Repository 

The CASA Data Repository, and how to update geodetic and ephemeris data in CASA.

Each CASA distribution comes with a minimal repository of binary data that is required for CASA to function properly. This repository includes [Measures Tables](external-data.ipynb#The-casacore-Measures-tables "Measures Tables") that deal with the Earth Orientation Parameters (EOPs), reference frames, as well as ephemeris data. In particular the EOPs include predictions for the near future which drift until they are well determined. They are regularly updated in the CASA data repository at NRAO of which the user\'s CASA installation contains a snapshot. Other data that is updated less frequently is also stored in the repository, such as beam models, antenna and Jy/K correction tables, and the antenna configuration files for the CASA simulator. Many tasks such as tclean, simobserve, and calibration tasks, need an up-to-date data repository to work properly. Because of this, the data repository of the CASA distribution must be updated occasionally as well.

#### Standard Update
A snapshot of this repository is included in each tarball distribution of CASA, and in the casadata module for CASA 6+. The snapshot can be found here:

```
CASA 6 tarball:    
linux: casa-6.1.0-118/data@ -> lib/py/lib/python3.6/site-packages/casadata/__data__/
macOS: /Applications/CASA-6.1.0-118.app/Contents/data@ -> Frameworks/Python.framework/Versions/Current/lib/python3.6/site-packages/casadata/__data__

casa5 tarball:    
casa-release-5.7.0-134.el7/data/
```
Note that for CASA 6 build from source, the repository snapshot is taken from .casa/config.py under: datapath=['/some-directory/']. More developer information on the Data Repository for build versions of CASA can be found in [this Knowledgebase Article](memo-series.ipynb#CASA-Data-Repository-for-developers-building-from-source "CASA Data Repository for developers").

The owner (in the Linux filesystem sense) of the CASA distribution can update the data repository to the latest available version by running from within CASA:

```
CASA <2>: ! update-data 
```

or (CASA 6 only) from the shell prompt outside CASA:
```
bash$ cd <release top directory>
bash$ bin/pip3 install --upgrade  --extra-index-url https://go.nrao.edu/pypi casadata
```

where ``<release top directory>`` is the top directory of the unpacked CASA tar file.

When using the modular pip-wheel CASA distribution, use:
```
bash$ pip3 install --upgrade  --extra-index-url https://go.nrao.edu/pypi casadata
```

Note that for CASA 6, these methods will update to the latest version of the casadata *module* which is only updated weekly.

To see the version number of the data repository that is included with your CASA version, please use the pip3 in the casa tarball distribution:
```
bash$ casa-6.1.0-118/bin/pip3 list |grep casadata (for linux)
bash$ CASA.app/Contents/MacOS/pip3 list |grep casadata (for mac)

or within CASA:
CASA <1>: import importlib_metadata
CASA <2>: importlib_metadata.version('casadata') Out[2]: '2020.8.3' 
```


#### Data Outside of Distribution
Instead of updating the CASA distribution directly. A copy of the runtime data can be fetched and maintained outside of the CASA distribution (or the Python packaged directory in the case of direct use of CASA packages). In the case of the tarball distribution, the user would:

```
bash$ cd <release top directory>
bash$ bin/python3 -m casatools --update-user-data
```

By default, this installs data in ``~/.casa/data``. This location can be changed by setting ``rundata`` in the CASA ``config.py`` file:

```
bash$ cat ~/.casa/config.py
rundata='~/my-casa-data'
bash$ 
```

running ``python3 -m casatools --update-user-data`` will use this ``rundata`` setting. To update data external to the CASA distribution (or the Python packaged directory in the case of direct use of CASA packages). This can be run again and it will only update the runtime data that has changed.

It is also possible to specify the directory on the command line with:

```
bash$ bin/python3 -m casatools --update-user-data=/tmp/my-cdata
```

This would install (or update) the runtime data in ``/tmp/my-cdata``. This directory could then be used by multiple users by setting ``rundata`` to ``/tmp/my-cdata``.


#### Direct Update of Distrubtion Data
As described above, the ``casadata`` package is updated by the CASA project weekly. This package can be updated, and this update cadence is sufficient for most users. However, the data that underlies these updates is updated daily. If users need updates more frequently than weekly, it is possible to update the runtime data in place. It is important to note that if this is done the CASA 6 version number of the ``casadata`` package will no longer be useful for understanding which state of the runtime data is being used.

For CASA 6, updating the CASA tarball distribution to the very latest data can be done with:

```
bash$ cd <release top directory>
bash$ bin/python3 -m casatools --update-user-data=`find lib -name __data__`
```

For CASA 5, updating to the very latest version of the data can be done with:

```
bash$ cd <release top directory>/data
bash$ bin/casa-config --sh-exec update-data
```

#### Entire Data Repository
The runtime data is just a small subset of CASA's entire data repository. It is possible to use rsync to get the complete data repository:

```
rsync -avz rsync://casa-rsync.nrao.edu/casa-data-repository casa-data
```

This data repository contains runtime data, reference data and test data. However, the CASA project is currently (Fall 2020) working to better organize and partition this data so these instructions will probably stop working in the future.


#####  Recommended update intervals 

One of the above data repository update methods should be used regularly for all CASA installations which are currently in use for processing recent data. Legacy installations processing old data do not have to be updated because the relevant contents of the Measures Tables is not changing anymore for the more distant past.

For observatory use, the update period should not be longer than weekly in order to have the EOPs up-to-date for upcoming observations. The shortest reasonable update interval is daily and the recommended method is via rsync.

For offline data analysis use, the update period should not be longer than monthly. Weekly update is recommended.

Typically, the administrator of a CASA installation sets up a cron job to perform the update automatically.

 

 



***





### The casacore Measures tables 

This chapter describes the casacore Measures tables needed to perform accurate conversions of reference frames.

The casacore infrastructure includes classes to handle physical quantities with a reference frame, so-called \"Measures\". Each type of Measure has its own distinct class in casacore which is derived from the Measure base class. One of the main functionalilties provided by casacore w.r.t. Measures, is the conversion of Measures from one reference frame to another using the MeasConvert classes.

Many of the spectral, spatial, and time reference frames are time-dependent and require the knowledge of the outcome of ongoing monitoring measurements of properties of the Earth and astronomical objects by certain service observatories. This data is tabulated in a number of tables (*\"Measures Tables\"*) which are stored in the CASA data repository in the subdirectory *\"geodetic\"*. A snapshot of this repository is included in each tarball distribution of CASA and in the casadata module for CASA6+. It can be updated to the latest status as described [here](external-data.ipynb#casa-data-repository).

The CASA data repository is updated daily based on the daily refinement of the geodetic information from the relevant services like the International Earth Rotation and Reference Systems Service (IERS). Strictly speaking, the Measures tables are part of the casacore infrastructure which is developed by NRAO, ESO, NAOJ, CSIRO, and ASTRON. In order to keep the repository consistent between the partners, the Measures tables are initially created at a single institution (ASTRON) and then copied into the NRAO CASA data repository from where all CASA users can retrieve them. As of March 2020, the update of the NRAO CASA copy of the Measures tables in \"geodetic\" and the planetary ephemerides in directory \"ephemerides\" takes place every day between 18 h UTC and 19 h UTC via two redundant servers at ESO (Garching).

The following list describes the individual Tables in subdirectory \"geodetic\":

* **IERSeop2000:**    The IERS EOP2000C04_05 Earth Orientation Parameters 
using the precession/nutation model \"IAU2000\" (files eopc04_IAU2000.xx)
* **IERSeop97**:    The IERS EOPC04_05 Earth Orientation Parameters using the precession/nutation model \"IAU 1980\" (files eopc04.xx)
* **IERSpredict:** IERS Earth Orientation Data predicted from NEOS (from file <ftp://ftp.iers.org/products/eop/rapid/daily/finals.daily>)
* **IGRF:**    International Geomagnetic Reference Field Schmidt [semi-normalised spherical harmonic coefficients](https://www.ngdc.noaa.gov/IAGA/vmod/coeffs/). (Note that this still uses IGRF12. An update to IGRF13 is underway.)
* IMF (not a Measures Table proper, access not integreated in Measures framework):    Historical interplanetary magnetic field data until MJD 52618 (December 2002).
* KpApF107 (not a Measures Table proper, access not integreated in Measures framework): Historical geomagnetic and solar activity indices until MJD 54921 (April 2009)
* **Observatories:** Table of the official locations of radio observatories. Maintained by the CASA consortium.
* SCHED_locations (not a Measures Table proper, access not integreated in Measures framework):    VLBI station locations
* **TAI_UTC:** TAI_UTC difference (i.e. leap second information) obtained from USNO

 

Measures Tables in the directory \"ephemerides\":

* **DE200**:    Historical JPL Planetary ephemeris DE200 used for Astronomical Almanach from 1984 to 2003 (from <ftp://ssd.jpl.nasa.gov/pub/eph/planets/ascii/de200>)
* **DE405**:    JPL Planetary ephemeris DE405; includes nutations and librations; referred to the ICRS (from <ftp://ssd.jpl.nasa.gov/pub/eph/planets/ascii/de405>)

***





### The JPL-Horizons ephemeris tables 

This chapter describes the ephemeris tables for a selection of the solar system objects from JPL-Horizons database.

The emphemeris data tables for the selected solar system objects are generated from the JPL Horizons system\'s on-line solar system data and ephemeris computation service ([https://ssd.jpl.nasa.gov/?horizons )](https://ssd.jpl.nasa.gov/?horizons "JPL Horizons"). These are primarily used to determine flux models for the solar system objects used in **setjy** task.  These tables are stored as CASA tables in the  CASA data repository under ephemerides/JPL-Horizons. The current ephemeris tables cover ephemerides until December 31, 2030 for those objects officially supported in **setjy**. Available objects, which include major planets, satellites, and asteroids, are: Mercury, **Venus**, **Mars**, **Jupiter**, Saturn, **Uranus**, **Neptune**, Pluto, **Io**, **Europa**, **Ganymede**, **Callisto**, **Titan**, **Ceres**, **Vesta**, **Pallas**, **Juno**, **Lutetia**, Sun and Moon (the objects in bold are those supported in \'Butler-JPL-Horizons 2012\' standard in **setjy**.). 

The format of the table name of these tables is *objectname*\_*startMJD*\_*endMJD*\_J2000.tab These tables required by **setjy** task are included in the data directory in the CASA distribution. The available tables can be listed by the following commands:

 

```
#In CASA6

CASA <1>: import glob

CASA <2>: jpldatapath=os.getenv('CASAPATH').split(' ')[0]+'/data/ephemerides/JPL-Horizons/*J2000.tab'

CASA <3>: glob.glob(jpldatapath)
```

 

The following data are retrieved from the JPL-Horizons system (the nubmer in the parentheses indicates the column number listed in the JPL-Horizons system). One should refer https://ssd.jpl.nasa.gov/?horizons_doc for the detailed descreption of each of these quantities.

  ------------------------------- ------------ ------------------------ ---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------- -----------------------
  Quantities                      column no.   Unit/format              Descrition                                                                                                                                                                         column label
  Date                            n.a.         YYYY-MM-DD HH:MM                                                                                                                                                                                            Date\_\_(UT)\_\_HR:MN
  Astrometric RA & DEC            1            degrees                  Astrometric RA and Dec with respect to the observer\'s location (GEOCETRIC)                                                                                                        R.A.\_(ICRF)\_DEC
  Observer sub-long& sub-lat      14           degrees                  Apparent planetodetic (\"geodetic\") longitude and latitude of the center of the target seen by the OBSERVER at print-time   ob-lon, ob-lat
  Solar sub-long & sub-lat        15           degrees                  Apparent planetodetic (\"geodetic\") longitude and latitude of the Sun seen by the OBSERVER at print-time                    Sl-lon, Sl-lat
   North Pole Pos. ang. & dist.    17          degrees and arcseconds    Target\'s North Pole position angle and angular distance from the \"sub-observer\" point                                                                                           NP.ang, NP.ds
   Helio range & range rate        19           AU, km/s                Heliocentric range (r) and range-rate (rdot)                                                                                                                                        r, rdot
   Observer range & range rate     20          AU, km/s                  Range (delta) and range-rate (deldot) of the target center with respect to the observer                                                                                            delta, dedot
   S-T-O angle                     24           degrees                 Sun-Target-Observer angle                                                                                                                                                           S-T-O
  ------------------------------- ------------ ------------------------ ---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------- -----------------------

 The script, request.py (located previously in recipes.ephemerides in CASA5, TBD for CASA6) can be used to retrieve the ephemeris data from the JPL-Horizons system via e-mail (See also [Manipulate Ephemeris Objects page](ephemeris_data.ipynb#Manipulate-Ephemeris-Objects "Manipulate Ephemeris Objects")). Further, the saved email text file is converted to a CASA table format using JPLephem_reader2.

 

```
#In CASA6

CASA <5>: from casatasks.private import JPLephem_reader2 as jplreader

CASA <6>: outdict = jplreader.readJPLephem('titan-jpl-horizons-ephem.eml')
opened the file=titan-jpl-horizons-ephem.eml

CASA <7>: jplreader.ephem_dict_to_table(outdict,'Titan_test_ephem.tab')
Got nrows = 3653 from MJD
```

The converted table contains following columns.

  ------------- ------------- ------------------------------------------------
  Column name   unit/format   description
  MJD           day           modified Julian date
  RA            degree         atrometric right acension in ICRF/J2000 frame
  DEC           degree         astrometric declination in ICRF/J2000 frame
  Rho           AU            Geocentric distance
  RadVal        AU/d          Geocentric distance rate
  NP_ang        degree        North pole position angle
  NP_dist       degree        North pole angular distance
  DiskLong      degree        Sub-observer longitude
  DiskLat       degree        Sub-observer latitude
  Sl_lon        degree        Sub-Solar longitude
  Sl_lat        degree        Sub-Solar latitude
  r             AU            heliocentric distance
  rdot          km/s          heliocentric distance rate
  phang         degree        phase angle
  ------------- ------------- ------------------------------------------------

 

There are currently no items in this folder.



***





### Array configuration files 

Array configuration files for various telescopes can be used in the CASA simulator

Array configuration files for various telescopes are distributed with each CASA release. These configuration files can be used to define the telescope for simulator tools and tasks. Currently, configuration files for the following telescopes are available in CASA:

-   ALMA / 12m Array
-   ALMA / 7m ACA
-   VLA
-   VLBA
-   Next-Generation VLA (reference design)
-   ATCA
-   MeerKat
-   PdBI (pre-NOEMA)
-   WSRT
-   SMA
-   Carma

The full list of antenna configurations can be found in the [CASA Guides on Simulations](https://casaguides.nrao.edu/index.php?title=Antenna_Configurations_Models_in_CASA_Cycle6).

One can also locate the directory with the configurations in the CASA distribution and then list the configuration files, using the following commands in CASA:

    CASA <>: print os.getenv('CASAPATH').split(' ')[0] + '/data/alma/simmos/'
    /home/casa/packages/RHEL7/release/casa-release-5.4.0-68/data/alma/simmos/

    CASA <>: ls /home/casa/packages/RHEL6/release/casa-release-5.4.0-68/data/alma/simmos/

<div>

If a configuration file is not distributed with CASA but retrieved elsewhere, then the configuration file can be called by explicitly writing the full path to the location of the configuration file in the antennalist paramter of the simulator tasks.

</div>

<div class="alert alert-info">
[**NOTE**: the most recent ALMA configuration files may not always be available in the most recent CASA version. ALMA configuration files for all cycles are available for download [here](https://almascience.nrao.edu/tools/casa-simulator). For the Next-Generation VLA reference design, the latest information can be found [here](https://ngvla.nrao.edu/page/tools).]
</div>
